In [1]:
# Include necessary imports
import os
import torch 
import pandas as pd
from torch.utils.data import DataLoader, TensorDataset
from music21 import *
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from torch.nn.utils.rnn import pad_sequence

In [2]:
# Preprocess the data

folder_path = 'Data/'
test = []
train = []
validation = []
for dirname in os.listdir(folder_path):
    if dirname != '.DS_Store':
        for filename in os.listdir(folder_path + dirname):
            if filename != '.ipynb_checkpoints':
                df = pd.read_csv(folder_path + dirname + '/' + filename)
                transposed_df = df.transpose()
                if dirname == 'test':
                    test.append(transposed_df)
                if dirname == 'train':
                    train.append(transposed_df)
                if dirname == 'valid':
                    validation.append(transposed_df)

# Model

In [71]:
class Model(torch.nn.Module):
    def __init__(self, input_size, output_size, hidden_dim=40, n_layers=2, dropout_rate=.5):
        super(Model, self).__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.lstm = torch.nn.LSTM(input_size, hidden_dim, n_layers, batch_first=True, dropout=dropout_rate)
        self.fc = torch.nn.Linear(hidden_dim, output_size)
        self.dropout = torch.nn.Dropout(dropout_rate)
        
    def forward(self, x, hidden=None):
        lstm_output, (h,c) = self.lstm(x, hidden)
        lstm_output = self.dropout(lstm_output)
        model_output = self.fc(lstm_output)
        return model_output

# Train

In [76]:
# def train_model(model, optimizer, criterion, num_epochs):
#     for epoch in range(num_epochs):
#         model.train()
#         for song in train:
#             song = key_transposition(song)
#             melody = torch.tensor(normalize_array(song.iloc[0].values.reshape(-1,1)), dtype=torch.float32).unsqueeze(0).reshape(1,song.shape[1],1)
#             harmonies = torch.tensor(normalize_array(song.iloc[1:].values.T), dtype=torch.float32).unsqueeze(0)
       
#             optimizer.zero_grad()
#             output = model(train_melody)
#             loss = criterion(output, train_harmonies)
#             loss.backward()
#             optimizer.step()
#             model.eval()
        
#         with torch.no_grad(): 
#             val_output = model(val_melody) 
#             val_loss = criterion(val_output, val_harmonies)
            
#         if (epoch + 1) % 100 == 0:
#             print("Epoch: ", epoch, "Loss: ", loss, "Validation Loss: ", val_loss)
            
def key_transposition(array):
    transpose_val = 48 - array.iloc[3][0]
    array += transpose_val
    return array

# Min-Max normalization technique
def normalize_array(array):
    X_std = (array - 1) / (88 - 1)
    return X_std       
            
def train_model(model, optimizer, criterion, num_epochs):
    # train songs
    for song_index, song in enumerate(train):
        print(f"Training on song {song_index + 1}")
        song = key_transposition(song)
        
        melody_and_bass = []
        melody_and_bass.append(song.iloc[0])
        melody_and_bass.append(song.iloc[3])
        melody_and_bass=pd.DataFrame(melody_and_bass)
#         print((melody_and_bass))
#         print(type(song.iloc[1:]))
#         print(song.iloc[0] + song.iloc[3])
#         melody = torch.tensor(normalize_array(melody_and_bass.values.T), dtype=torch.float32).reshape(1, melody_and_bass.shape[1], 2)
        bass = torch.tensor(normalize_array(song.iloc[3].values.reshape(-1, 1)), dtype=torch.float32).unsqueeze(0).reshape(1, song.shape[1], 1)
        harmonies = torch.tensor(normalize_array(song.iloc[:3].values.T), dtype=torch.float32).unsqueeze(0)
        
        for epoch in range(num_epochs):
            model.train()
            optimizer.zero_grad()
            output = model(bass)
            loss = criterion(output, harmonies)
            loss.backward()
            optimizer.step()

            if (epoch + 1) % 100 == 0:
                print(f"Song {song_index + 1}, Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item()}")
                if torch.isnan(loss):
                    print("harmonies: ", harmonies)
                    print("melody:" , melody)
                    print("output: ", output)
                    

        model.eval()
        total_val_loss = 0
        
#         # validation songs
#         with torch.no_grad():
#             for val_song in validation_loader:
#                 val_song = key_transposition(val_song)
#                 val_melody = torch.tensor(normalize_array(val_song.iloc[0].values.reshape(-1, 1)), dtype=torch.float32).unsqueeze(0).reshape(1, val_song.shape[1], 1)
#                 val_harmonies = torch.tensor(normalize_array(val_song.iloc[1:].values.T), dtype=torch.float32).unsqueeze(0)
                
#                 val_output = model(val_melody)
#                 val_loss = criterion(val_output, val_harmonies)
#                 total_val_loss += val_loss.item()

#         average_val_loss = total_val_loss / len(validation)
#         print(f"Validation Loss after song {song_index + 1}: {average_val_loss}")

In [ ]:
criterion = torch.nn.MSELoss()
num_epochs = 5000
index = 0
model = Model(1, 3)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# batch_size = 16

# train_loader = DataLoader(torch.tensor(train), batch_size=batch_size)
# validation_loader = DataLoader(torch.tensor(validation), batch_size=batch_size)

train_model(model, optimizer, criterion, num_epochs)


# train_dataset = TensorDataset(torch.tensor(train))
# validation_dataset = TensorDataset(torch.tensor(validation))

# for song in train[:10]:
#     index = index + 1
#     print("song", index)
#     song = key_transposition(song)
# #     song = train[0] # REMOVE
#     melody = torch.tensor(normalize_array(song.iloc[0].values.reshape(-1,1)), dtype=torch.float32).unsqueeze(0).reshape(1,song.shape[1],1)
#     harmonies = torch.tensor(normalize_array(song.iloc[1:].values.T), dtype=torch.float32).unsqueeze(0)
    
#     train_model(model, melody, harmonies, optimizer, criterion, epoch)

Training on song 1
Song 1, Epoch 100/5000, Loss: 0.011602784506976604
Song 1, Epoch 200/5000, Loss: 0.005373675841838121
Song 1, Epoch 300/5000, Loss: 0.0029771258123219013
Song 1, Epoch 400/5000, Loss: 0.0019958887714892626
Song 1, Epoch 500/5000, Loss: 0.001237217104062438
Song 1, Epoch 600/5000, Loss: 0.000971946690697223
Song 1, Epoch 700/5000, Loss: 0.0007642423734068871
Song 1, Epoch 800/5000, Loss: 0.0007377205183729529
Song 1, Epoch 900/5000, Loss: 0.0006950875977054238
Song 1, Epoch 1000/5000, Loss: 0.0006802527932450175
Song 1, Epoch 1100/5000, Loss: 0.0006826668977737427
Song 1, Epoch 1200/5000, Loss: 0.0006271681049838662
Song 1, Epoch 1300/5000, Loss: 0.0006516118883155286
Song 1, Epoch 1400/5000, Loss: 0.0006341449916362762
Song 1, Epoch 1500/5000, Loss: 0.0006412179791368544
Song 1, Epoch 1600/5000, Loss: 0.0006330201867967844
Song 1, Epoch 1700/5000, Loss: 0.0006270651938393712
Song 1, Epoch 1800/5000, Loss: 0.0006330730393528938
Song 1, Epoch 1900/5000, Loss: 0.0006320

# Hyperparameter Tuning

In [ ]:
learning = [0.01, .001]
n_layers= [1,2,3]
hidden_dim = [20, 40, 50]
epochs= [5000, 10000]
best_loss = float('inf')
best_params = {}


for LR in learning:
    for n_layer in n_layers:
        for epoch in epochs:
            for dims in hidden_dim:
                print(f"Training with LR={LR} and n_layers={n_layer} and epochs={epoch} and hidden_dims={dims}")
                model = Model(input_size=1, output_size=harmonies.shape[2], n_layers=n_layer, hidden_dim=dims)
                optimizer = torch.optim.Adam(model.parameters(), lr=LR)
                train_model(model, melody, harmonies, optimizer, criterion, epoch)
                with torch.no_grad():
                    output = model(melody)
                    loss = criterion(output, harmonies)
                    print(f"Final Loss: {loss.item()}")        
                # Keep track of the best model (with lowest loss)
                if loss.item() < best_loss:
                    best_loss = loss.item()
                    best_params = {'learning': LR, 'n_layers': n_layer, 'epochs': epoch, 'hidden_dim': dims}
print("BEST: ", best_params)

In [ ]:
# tiny = pd.DataFrame([[67,62,59,43], [68,62,59,43]]).transpose()
# melody = torch.tensor(tiny.iloc[0], dtype=torch.float32).unsqueeze(0).reshape(1,2,1)
# harmonies = torch.transpose(torch.tensor(tiny.iloc[1:].values, dtype=torch.float32),0,1).unsqueeze(0)
# model = Model(1, harmonies.shape[2])
# optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
# criterion = torch.nn.MSELoss()
# train_model(model, melody, harmonies, optimizer, criterion, 10000)

In [ ]:
def inverse_array(array):
    X_scaled = array * (88 - 1) + 1
    return X_scaled

def midi_to_note(part):
    result = stream.Part()
    count = 1
    prev = part[0]
    for i in range(1, len(part)):
        pitch = part[i]
        curr = round(pitch)
        if curr == prev:
            count += 1
        else:
            this_note = note.Note(prev, quarterLength=count/4)
            result.append(this_note)
            count = 1
        prev = curr
    this_note = note.Note(prev, quarterLength=count/4)
    result.append(this_note)
    return result

def output_to_sheet_music(melody, result):
    result_numpy = result.detach().numpy()
    inversed = inverse_array(np.squeeze(result_numpy)).T
    
    score = stream.Score()
    melody_part = midi_to_note(melody)
    
    alto_notes = inversed[0]
    tenor_notes = inversed[1]
    bass_notes = inversed[2]  
    
    alto_part = midi_to_note(alto_notes)
    tenor_part = midi_to_note(tenor_notes)
    bass_part = midi_to_note(bass_notes)

    score.append(melody_part)
    score.append(alto_part)
    score.append(tenor_part)
    score.append(bass_part)
    score.show('midi')
    score.write('musicxml', 'output.xml')

In [ ]:
test_song = key_transposition(test[0])
melody = test_song.iloc[0]
# print("melody is: ", torch.tensor(normalize_array(song.iloc[0].values.reshape(-1,1)), dtype=torch.float32).unsqueeze(0).reshape(1,song.shape[1],1))
# harmonies = torch.tensor(normalize_array(test[0].iloc[1:].values.T), dtype=torch.float32).unsqueeze(0)
result = model(torch.tensor(normalize_array(test_song.iloc[0].values.reshape(-1,1)), dtype=torch.float32).unsqueeze(0).reshape(1,test[0].shape[1],1))
output_to_sheet_music(melody, result)

In [ ]:
# Finetune (hyperparameters, move around test data (refer to notes), etc)

In [ ]:
# Test with new data + evaluate

In [ ]:
# Make any other changes

In [ ]:
# Sheet music + audio (musicAI)

In [ ]:
# Create new models if time permits (follow steps 3 - 7)

In [ ]:
# Compare models

In [ ]:
# Front end ** if time permits
# - Interactive sheet music
# - musescore front end??

In [ ]:
# real one and generated compare
# train on all songs + test on a different song
# measure the test loss not just the training loss


In [13]:
import numpy as np

array1 = np.array([1, 2, 3])
array2 = np.array([4, 5, 6])

result = np.concatenate((array1, array2))

print(result)

[1 2 3 4 5 6]
